In [ ]:
import math

def ra_dec_to_deg(ra_hours, ra_minutes, ra_seconds, dec_degrees, dec_minutes, dec_seconds):
    # 转换时角为角度
    ra_degrees = (ra_hours + ra_minutes/60 + ra_seconds/3600) * 15

    # 转换赤纬为角度
    sign = -1 if dec_degrees < 0 else 1
    dec_degrees = abs(dec_degrees)
    dec_degrees += dec_minutes/60 + dec_seconds/3600
    dec_degrees *= sign

    return ra_degrees, dec_degrees



def J2000ToEqu(mjd, ras2000, dec2000):
    obsite_la = 29.357656306
    obsite_lo = 100.138794639
    obsite_he = 4376.37
    x, y, z, x2000, y2000, z2000 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    p11, p12, p13, p21, p22, p23, p31, p32, p33 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    zeta, zee, theta, tu, tu2, tu3 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
    sinzeta, coszeta, sinzee, coszee, sintheta, costheta = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

    D2R = math.pi / 180.0
    R2D = 180.0 / math.pi

    ras2000 *= D2R
    dec2000 *= D2R

    tu = (mjd - 51544.5) / 36525.0
    tu2 = tu * tu
    tu3 = tu2 * tu
    zeta = 0.0111808609 * tu + 1.463556e-6 * tu2 + 8.725677e-8 * tu3
    zee = 0.0111808609 * tu + 5.307158e-6 * tu2 + 8.825063e-8 * tu3
    theta = 0.0097171735 * tu - 2.068458e-6 * tu2 - 2.028121e-7 * tu3

    sinzeta = math.sin(zeta)
    coszeta = math.cos(zeta)
    sinzee = math.sin(zee)
    coszee = math.cos(zee)
    sintheta = math.sin(theta)
    costheta = math.cos(theta)

    p11 = coszeta * costheta * coszee - sinzeta * sinzee
    p12 = -sinzeta * costheta * coszee - coszeta * sinzee
    p13 = -sintheta * coszee
    p21 = coszeta * costheta * sinzee + sinzeta * coszee
    p22 = -sinzeta * costheta * sinzee + coszeta * coszee
    p23 = -sintheta * sinzee
    p31 = coszeta * sintheta
    p32 = -sinzeta * sintheta
    p33 =  costheta

    x2000 = math.cos(dec2000) * math.cos(ras2000)
    y2000 = math.cos(dec2000) * math.sin(ras2000)
    z2000 = math.sin(dec2000)
    x = p11 * x2000 + p12 * y2000 + p13 * z2000
    y = p21 * x2000 + p22 * y2000 + p23 * z2000
    z = p31 * x2000 + p32 * y2000 + p33 * z2000
    ras = math.atan2(y, x) * R2D
    if (ras < 0.0): ras += 360.0
    dec = math.asin(z) * R2D
    
    return ras,dec

def EquToHor(mjd, ras, dec):
    obsite_la = 29.357656306
    obsite_lo = 100.138794639
    obsite_he = 4376.37
    D2R = math.pi / 180.0
    R2D = 180.0 / math.pi
    
    # Convert to radians
    ras = ras * D2R
    dec = dec * D2R
    obsite_la = obsite_la * D2R

    # Calculate Local Mean Sidereal Time (LMST)
    tu = (mjd - 51544.5) / 36525.0
    gmst0 = 0.279057273263889 + 100.002139037801 * tu + 1.07759259e-6 * tu * tu - 7.1759259e-11 * tu * tu * tu
    lmst = gmst0 + mjd + obsite_lo / 360.0
    lmst = (lmst - math.floor(lmst)) * 360.0
    
    # Calculate Hour Angle (H)
    H = lmst - ras

    # Calculate trigonometric values
    sH = math.sin(H)
    sdec = math.sin(dec)
    sphi = math.sin(obsite_la)
    cH = math.cos(H)
    cdec = math.cos(dec)
    cphi = math.cos(obsite_la)

    # Calculate zenith and azimuth
    zen = math.acos(sphi * sdec + cphi * cdec * cH) * R2D
    sazi = (-cdec * sH)
    cazi = (cphi * sdec - sphi * cdec * cH)
    azi = math.atan2(sazi, cazi) * R2D

    # Ensure azimuth is in the range [-180, 180]
    if azi >= 180.0:
        azi -= 360.0
    if azi < -180.0:
        azi += 360.0

    return zen, azi

def rangeAngle(azi):
    if(azi < 0): azi+=360
    if(azi > 360): azi-=360
    return azi

def wcdacoor(azi):
	azi = 60.558 - azi
	return rangeAngle(azi)

: 